In [1]:
a_file = open("ref1.txt", "r")
def cloning(li):
    new_li=[]
    for item in li:
        new_li.append(item)
    return new_li    

layer1 = []
layer2 = []


for line in a_file:
    stripped_line = line.strip()
    #print(stripped_line)
    line_list = stripped_line.split()
    if line_list[-1]!='<eos>':
        layer2.append(line_list)
    else:
        layer2.append(line_list[:len(line_list)-1])
        
    #print(layer2)
    if line_list[-1]=='<eos>':
       
        layer1.append(cloning(layer2))
        print(layer1)
        layer2.clear()
        #print(layer1)
        
#print(layer1)       

#layer1 ->[[['I', 'want', 'food'], ['I', 'want', 'the', 'food', '<eos>']]]
#layer1 ->[[]] ->[layer2]
#layer2 ->[['I', 'came', '<eos>']]
#layer1 ->[[['I', 'came', '<eos>']]]
#layer1 ->[[],[]]
#

[[['I', 'want', 'food']]]
[[['I', 'want', 'food']], [['I', 'came']]]
[[['I', 'want', 'food']], [['I', 'came']], [['I', 'stay', 'in', 'pune']]]
[[['I', 'want', 'food']], [['I', 'came']], [['I', 'stay', 'in', 'pune']], [['What', 'is', 'your', 'name?']]]
[[['I', 'want', 'food']], [['I', 'came']], [['I', 'stay', 'in', 'pune']], [['What', 'is', 'your', 'name?']], [['Where', 'is', 'the', 'bank?']]]
[[['I', 'want', 'food']], [['I', 'came']], [['I', 'stay', 'in', 'pune']], [['What', 'is', 'your', 'name?']], [['Where', 'is', 'the', 'bank?']], [['I', 'did', 'not', 'understand']]]
[[['I', 'want', 'food']], [['I', 'came']], [['I', 'stay', 'in', 'pune']], [['What', 'is', 'your', 'name?']], [['Where', 'is', 'the', 'bank?']], [['I', 'did', 'not', 'understand']], [['Father', 'is', 'a', 'clerk']]]
[[['I', 'want', 'food']], [['I', 'came']], [['I', 'stay', 'in', 'pune']], [['What', 'is', 'your', 'name?']], [['Where', 'is', 'the', 'bank?']], [['I', 'did', 'not', 'understand']], [['Father', 'is', 'a', 'cle

IndexError: list index out of range

In [ ]:
b_file = open("ref.txt", "r")


list_of_lists_ = []

for line in b_file:
    stripped_line = line.strip()
    line_list = stripped_line.split()
    list_of_lists_.append(line_list)


b_file.close()


print(list_of_lists_)

In [ ]:
import collections
import math



def _get_ngrams(segment, max_order):
  """Extracts all n-grams upto a given maximum order from an input segment.
  Args:
    segment: text segment from which n-grams will be extracted.
    max_order: maximum length in tokens of the n-grams returned by this
        methods.
  Returns:
    The Counter containing all n-grams upto max_order in segment
    with a count of how many times each n-gram occurred.
  """
  ngram_counts = collections.Counter()
  for order in range(1, max_order + 1):
    for i in range(0, len(segment) - order + 1):
      ngram = tuple(segment[i:i+order])
      ngram_counts[ngram] += 1
  return ngram_counts


def compute_bleu(reference_corpus, translation_corpus, max_order=4,
                 smooth=False):
  """Computes BLEU score of translated segments against one or more references.
  Args:
    reference_corpus: list of lists of references for each translation. Each
        reference should be tokenized into a list of tokens.
    translation_corpus: list of translations to score. Each translation
        should be tokenized into a list of tokens.
    max_order: Maximum n-gram order to use when computing BLEU score.
    smooth: Whether or not to apply Lin et al. 2004 smoothing.
  Returns:
    3-Tuple with the BLEU score, n-gram precisions, geometric mean of n-gram
    precisions and brevity penalty.
  """
  matches_by_order = [0] * max_order
  possible_matches_by_order = [0] * max_order
  reference_length = 0
  translation_length = 0
  for (references, translation) in zip(reference_corpus,
                                       translation_corpus):
    reference_length += min(len(r) for r in references)
    translation_length += len(translation)

    merged_ref_ngram_counts = collections.Counter()
    for reference in references:
      merged_ref_ngram_counts |= _get_ngrams(reference, max_order)
    translation_ngram_counts = _get_ngrams(translation, max_order)
    overlap = translation_ngram_counts & merged_ref_ngram_counts
    for ngram in overlap:
      matches_by_order[len(ngram)-1] += overlap[ngram]
    for order in range(1, max_order+1):
      possible_matches = len(translation) - order + 1
      if possible_matches > 0:
        possible_matches_by_order[order-1] += possible_matches

  precisions = [0] * max_order
  for i in range(0, max_order):
    if smooth:
      precisions[i] = ((matches_by_order[i] + 1.) /
                       (possible_matches_by_order[i] + 1.))
    else:
      if possible_matches_by_order[i] > 0:
        precisions[i] = (float(matches_by_order[i]) /
                         possible_matches_by_order[i])
      else:
        precisions[i] = 0.0

  if min(precisions) > 0:
    p_log_sum = sum((1. / max_order) * math.log(p) for p in precisions)
    geo_mean = math.exp(p_log_sum)
  else:
    geo_mean = 0

  ratio = float(translation_length) / reference_length

  if ratio > 1.0:
    bp = 1.
  else:
    bp = math.exp(1 - 1. / ratio)

  bleu = geo_mean * bp
  return (bleu, precisions, bp, ratio, translation_length, reference_length)

In [ ]:
compute_bleu(layer1,list_of_lists_,4,False)